<b><font color="SteelBlue" size="+3">Classifiez automatiquement des biens de consommation</font></b>

# Introduction

Ce notebook est la suite de celui sur le traitement des descriptions. On récupère les données nettoyées pour les catégories des produits.

## Imports

In [1]:
# Chargement des librairies
# Standard
import os

# Data Manipulation and Analysis
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Image Processing
from matplotlib.image import imread
from PIL import Image as PilImage

# Deep Learning - TensorFlow/Keras
os.environ["KERAS_BACKEND"] = "torch"
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import preprocess_input as preprocess_vgg16
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input as preprocess_inceptionresnetv2
from tensorflow.keras.applications.densenet import preprocess_input as preprocess_densenet201
from tensorflow.keras.utils import to_categorical

# Custom Modules
from Deveau_Estelle_4_IMG_func_042024 import *

## Data

In [2]:
os.listdir()

['.ipynb_checkpoints',
 'data',
 'Deveau_Estelle_1_notebook_txt_faisabilite_042024.ipynb',
 'Deveau_Estelle_2_NLP_func_042024.py',
 'Deveau_Estelle_3_notebook_img_faisabilite_042024.ipynb',
 'Deveau_Estelle_4_IMG_func_042024.py',
 'Deveau_Estelle_5_notebook_img_classification_042024.ipynb',
 'model_best.weights.h5',
 'model_densenet201_best.weights.h5',
 'model_inceptionresnetv2_best.weights.h5',
 'model_vgg16_best.weights.h5',
 'OC_projet6.pptx',
 'pythonProject',
 'svg',
 'Weather_Images_CNN_Transfer_Learning_Stage_1_feasibility_V1.0.ipynb',
 'Weather_Images_CNN_Transfer_Learning_Stage_2_supervised_classification_V1.0.ipynb',
 '__pycache__']

In [3]:
os.listdir("data/Cleaned/")

['data.csv']

In [4]:
path     = "./data/Cleaned/"
filename = "data.csv"

In [5]:
data = pd.read_csv(path + filename)

In [6]:
path     = "./data/Sources/Images/"
list_photos = [file for file in os.listdir(path)]
print(len(list_photos))

1050


# Classification supervisée

In [7]:
cols_to_keep = ['uniq_id', 'product_category']
df = data[cols_to_keep]
df.head(5)

,uniq_id,product_category
0,55b85ea15a1536d46b7190ad6fff8ce7,Home Furnishing
1,7b72c92c2f6c40268628ec5f14c6d590,Baby Care
2,64d5d4a258243731dc7bbb1eef49ad74,Baby Care
3,d4684dcdc759dd9cdf41504698d737d8,Home Furnishing
4,6325b6870c54cd47be6ebfbffa620ec7,Home Furnishing


In [8]:
image_paths = [f"{path}{uid}.jpg" for uid in df['uniq_id']]

# Encodage des catégories
label_encoder = LabelEncoder()
categories_encoded = label_encoder.fit_transform(df['product_category'])
category_names = label_encoder.classes_

## Séparation des jeux de données (train, val, test)

In [9]:
y = to_categorical(categories_encoded, 7)

In [10]:
y[:5]

array([[0., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.]])

In [11]:
# Diviser les chemins d'image et les étiquettes
paths_train, paths_test, y_train, y_test = train_test_split(image_paths, y, test_size=0.2, random_state=42)
paths_train, paths_val, y_train, y_val = train_test_split(paths_train, y_train, test_size=0.25, random_state=42)

In [12]:
y_test[:5]

array([[0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.]])

In [13]:
# Dictionnaire pour stocker les résultats
results = {}

## Evaluation des modèles sélectionnés

In [14]:
# Définir le seuil à un niveau supérieur spécifique
PilImage.MAX_IMAGE_PIXELS = 100000000

### VGG16

In [15]:
# Préparation des données
X_train_vgg16, X_val_vgg16, X_test_vgg16 = prepare_data(paths_train, paths_val, paths_test, preprocess_vgg16,
                                                        target_size=(224, 224, 3))

In [16]:
# Création du modèle
model_vgg16 = create_model_fct(base_model_name='VGG16')

In [17]:
# Entraînement du modèle
model_vgg16, history_vgg16, duration_vgg16 = train_model(model_vgg16, X_train_vgg16, y_train, X_val_vgg16, y_val,
                                                         model_save_path="./model_vgg16_best.weights.h5")

Epoch 1/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step - accuracy: 0.3030 - loss: 7.3882
Epoch 1: val_loss improved from inf to 1.72942, saving model to ./model_vgg16_best.weights.h5
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 343ms/step - accuracy: 0.3124 - loss: 7.2331 - val_accuracy: 0.6667 - val_loss: 1.7294
Epoch 2/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 242ms/step - accuracy: 0.6883 - loss: 2.1748
Epoch 2: val_loss improved from 1.72942 to 1.21963, saving model to ./model_vgg16_best.weights.h5
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 339ms/step - accuracy: 0.6878 - loss: 2.1779 - val_accuracy: 0.7524 - val_loss: 1.2196
Epoch 3/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 242ms/step - accuracy: 0.7500 - loss: 1.7212
Epoch 3: val_loss improved from 1.21963 to 1.03841, saving model to ./model_vgg16_best.weights.h5
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 339ms/step - accuracy: 0.7490 - loss: 1.7149 - val_accuracy: 0.7857 - val_loss: 1.0384
Epoch 4/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step - accuracy: 0.8108 - loss: 1.1967
Epoch 4: val_lo

In [18]:
# Évaluation du modèle
loss_vgg16, accuracy_vgg16, ari_score_vgg16 = evaluate_model(model_vgg16, X_train_vgg16, y_train, X_val_vgg16,
                                                             y_val, X_test_vgg16, y_test, best_weights_path="./model_vgg16_best.weights.h5")

20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 124ms/step - accuracy: 0.9991 - loss: 0.0208
Training Accuracy after last epoch: 0.9984

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 116ms/step - accuracy: 0.8352 - loss: 0.6420
Test Accuracy after last epoch: 0.8333

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 117ms/step - accuracy: 0.8068 - loss: 0.8235
Validation Accuracy (best): 0.7810
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 117ms/step - accuracy: 0.8392 - loss: 0.7066
Test Accuracy (best): 0.8286
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 118ms/step
Adjusted Rand Index (ARI): 0.6706


### InceptionResNetV2

In [19]:
# Préparation des données
X_train_inceptionresnetv2, X_val_inceptionresnetv2, X_test_inceptionresnetv2 = prepare_data(paths_train, paths_val,
                                                                                            paths_test, preprocess_inceptionresnetv2,
                                                                                            target_size=(299, 299, 3))

In [20]:
# Création du modèle
model_inceptionresnetv2 = create_model_fct(base_model_name='InceptionResNetV2')

In [21]:
# Entraînement du modèle
model_iresnetv2, history_iressnetv2, duration_iresnetv2 = train_model(model_inceptionresnetv2, X_train_inceptionresnetv2,
                                                                      y_train, X_val_inceptionresnetv2, y_val,
                                                                      model_save_path="./model_inceptionresnetv2_best.weights.h5")

Epoch 1/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.3552 - loss: 2.0370
Epoch 1: val_loss improved from inf to 0.80850, saving model to ./model_inceptionresnetv2_best.weights.h5
10/10 ━━━━━━━━━━━━━━━━━━━━ 102s 10s/step - accuracy: 0.3688 - loss: 1.9979 - val_accuracy: 0.7619 - val_loss: 0.8085
Epoch 2/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.7891 - loss: 0.7449
Epoch 2: val_loss improved from 0.80850 to 0.57831, saving model to ./model_inceptionresnetv2_best.weights.h5
10/10 ━━━━━━━━━━━━━━━━━━━━ 103s 11s/step - accuracy: 0.7884 - loss: 0.7443 - val_accuracy: 0.7905 - val_loss: 0.5783
Epoch 3/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.8412 - loss: 0.5055
Epoch 3: val_loss improved from 0.57831 to 0.48666, saving model to ./model_inceptionresnetv2_best.weights.h5
10/10 ━━━━━━━━━━━━━━━━━━━━ 114s 12s/step - accuracy: 0.8412 - loss: 0.5078 - val_accuracy: 0.8429 - val_loss: 0.4867
Epoch 4/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 0.8645 - loss

In [22]:
# Évaluation du modèle
loss_inceptionresnetv2, accuracy_inceptionresnetv2, ari_score_inceptionresnetv2 = evaluate_model(model_inceptionresnetv2,
                                                                                                 X_train_inceptionresnetv2,
                                                                                                 y_train, X_val_inceptionresnetv2,
                                                                                                 y_val, X_test_inceptionresnetv2,
                                                                                                 y_test,
                                                                                                 best_weights_path="./model_inceptionresnetv2_best.weights.h5")

20/20 ━━━━━━━━━━━━━━━━━━━━ 100s 5s/step - accuracy: 0.9788 - loss: 0.0810
Training Accuracy after last epoch: 0.9794

7/7 ━━━━━━━━━━━━━━━━━━━━ 34s 5s/step - accuracy: 0.8656 - loss: 0.4448
Test Accuracy after last epoch: 0.8810

7/7 ━━━━━━━━━━━━━━━━━━━━ 34s 5s/step - accuracy: 0.8597 - loss: 0.3548
Validation Accuracy (best): 0.8667
7/7 ━━━━━━━━━━━━━━━━━━━━ 34s 5s/step - accuracy: 0.9046 - loss: 0.3397
Test Accuracy (best): 0.8810
7/7 ━━━━━━━━━━━━━━━━━━━━ 33s 5s/step
Adjusted Rand Index (ARI): 0.7695


### DenseNet201

In [23]:
# Préparation des données
X_train_densenet201, X_val_densenet201, X_test_densenet201 = prepare_data(paths_train, paths_val, paths_test,
                                                                          preprocess_densenet201, target_size=(224, 224, 3))


In [24]:
# Création du modèle
model_densenet201 = create_model_fct(base_model_name='DenseNet201')


In [ ]:
# Entraînement du modèle
model_densenet201, history_densenet201, duration_densenet201 = train_model(model_densenet201, X_train_densenet201, y_train,
                                                                           X_val_densenet201, y_val,
                                                                           model_save_path="./model_densenet201_best.weights.h5")


Epoch 1/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 26s/step - accuracy: 0.3310 - loss: 2.3257 
Epoch 1: val_loss improved from inf to 0.81878, saving model to ./model_densenet201_best.weights.h5
10/10 ━━━━━━━━━━━━━━━━━━━━ 349s 36s/step - accuracy: 0.3456 - loss: 2.2731 - val_accuracy: 0.7667 - val_loss: 0.8188
Epoch 2/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 26s/step - accuracy: 0.7903 - loss: 0.7069 
Epoch 2: val_loss improved from 0.81878 to 0.66513, saving model to ./model_densenet201_best.weights.h5
10/10 ━━━━━━━━━━━━━━━━━━━━ 349s 36s/step - accuracy: 0.7909 - loss: 0.7032 - val_accuracy: 0.7857 - val_loss: 0.6651
Epoch 3/50
 7/10 ━━━━━━━━━━━━━━━━━━━━ 1:13 25s/step - accuracy: 0.8427 - loss: 0.4921

In [ ]:
# Évaluation du modèle
loss_densenet201, accuracy_densenet201, ari_score_densenet201 = evaluate_model(model_densenet201, X_train_densenet201,
                                                                               y_train, X_val_densenet201,
                                                                                y_val, X_test_densenet201, y_test,
                                                                               best_weights_path="./model_densenet201_best.weights.h5")

### Comparaison des résultats

In [ ]:
# Initialisation de la liste pour stocker les résultats
data_results = []

# Fonction pour ajouter les résultats d'un modèle au DataFrame
def add_results(model_name, loss, accuracy, ari_score, duration):
    metrics = ['Loss', 'Accuracy', 'Adjusted Rand Score', 'Duration (en heures)']
    scores = [loss, accuracy, ari_score, (duration/3600)]
    for metric, score in zip(metrics, scores):
        data_results.append({
            'Model': model_name,
            'Metric': metric,
            'Score': score
        })

In [ ]:
add_results('VGG16', loss_vgg16, accuracy_vgg16, ari_score_vgg16, duration_vgg16)
add_results('InceptionResNetV2', loss_inceptionresnetv2, accuracy_inceptionresnetv2, ari_score_inceptionresnetv2, duration_iresnetv2)
add_results('DenseNet201', loss_densenet201, accuracy_densenet201, ari_score_densenet201, duration_densenet201)

data_metrics = pd.DataFrame(data_results)

In [ ]:
plot_model_performance(data_results)

# Optimisation

## Hyper paramètres

In [ ]:
# Entraînement du modèle
best_model, duration_best = test_hyperparameters(model_inceptionresnetv2, X_train_inceptionresnetv2, y_train, X_val_inceptionresnetv2,
                                                 y_val, model_save_path="./model_best.weights.h5")

In [ ]:
# Évaluation du modèle
loss_best, accuracy_best, ari_score_best = evaluate_model(best_model,
                                                          X_train_inceptionresnetv2,
                                                          y_train, X_val_inceptionresnetv2,
                                                          y_val, X_test_inceptionresnetv2,
                                                          y_test, best_weights_path="none")

In [ ]:
add_results('InceptionResNetV2(HP)', loss_best, accuracy_best, ari_score_best, duration_best)
data_metrics = pd.DataFrame(data_results)

In [ ]:
data_metrics.head(15)

In [ ]:
plot_model_performance(data_metrics)

## Data Augmentation